In [3]:

import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import BaggingClassifier,RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier


In [36]:
application_data= pd.read_csv("ApplicationData.csv")
application_data.head()

,Application_ID,Application_Date,Loan_Approval_Status,Age_at_Application,Marital_Status,Gender,Own_Rent_Home,Occupation_Professional_Category,Type_of_Business_Industry_of_Employment,Employment_Status,Time_at_Current_Employment_(Months),Time_Employed_with_Previous_Employer_(Months),Number_of_Dependents,Loan_Purpose,Monthly_Income,Government_Employee
0,C302100001,2015-09-15,Approved,39.0,Unmarried,Female,Own,NaN,Other,Full-Time,6.0,NaN,2.0,Home Improvement,1500.59,NaN
1,C302100002,2015-08-31,Approved,29.0,Unmarried,Female,Family,NaN,NaN,Full-Time,25.0,NaN,0.0,Debt Consolidation,3769.36,NaN
2,C302100003,2015-09-21,Approved,24.0,Unmarried,Male,Family,NaN,NaN,Full-Time,7.0,NaN,0.0,Home Improvement,2197.10,NaN
3,C302100004,2015-11-14,Approved,54.0,Unmarried,Male,Rent,NaN,NaN,Full-Time,105.0,NaN,4.0,Debt Consolidation,1094.45,False
4,C302100005,2015-11-18,Approved,30.0,Unmarried,Female,Family,NaN,NaN,Full-Time,60.0,NaN,1.0,Vacation,3798.40,True


In [7]:
application_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9492 entries, 0 to 9491
Data columns (total 16 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Application_ID                                 9492 non-null   object 
 1   Application_Date                               9492 non-null   object 
 2   Loan_Approval_Status                           9354 non-null   object 
 3   Age_at_Application                             9471 non-null   float64
 4   Marital_Status                                 7712 non-null   object 
 5   Gender                                         9488 non-null   object 
 6   Own_Rent_Home                                  8188 non-null   object 
 7   Occupation_Professional_Category               9361 non-null   object 
 8   Type_of_Business_Industry_of_Employment        7210 non-null   object 
 9   Employment_Status                              9357 

In [6]:
application_data.isnull().sum()

Application_ID                                      0
Application_Date                                    0
Loan_Approval_Status                              138
Age_at_Application                                 21
Marital_Status                                   1780
Gender                                              4
Own_Rent_Home                                    1304
Occupation_Professional_Category                  131
Type_of_Business_Industry_of_Employment          2282
Employment_Status                                 135
Time_at_Current_Employment_(Months)               232
Time_Employed_with_Previous_Employer_(Months)    9194
Number_of_Dependents                             1814
Loan_Purpose                                       42
Monthly_Income                                      0
Government_Employee                               181
dtype: int64

In [34]:
#selecting the columns to use for our model 
df_appli_selected = application_data.drop(columns=["Government_Employee","Application_Date"])
df_appli_selected

,Application_ID,Loan_Approval_Status,Age_at_Application,Marital_Status,Gender,Own_Rent_Home,Occupation_Professional_Category,Type_of_Business_Industry_of_Employment,Employment_Status,Time_at_Current_Employment_(Months),Time_Employed_with_Previous_Employer_(Months),Number_of_Dependents,Loan_Purpose,Monthly_Income
0,C302100001,Approved,39.0,Unmarried,Female,Own,NaN,Other,Full-Time,6.0,NaN,2.0,Home Improvement,1500.59
1,C302100002,Approved,29.0,Unmarried,Female,Family,NaN,NaN,Full-Time,25.0,NaN,0.0,Debt Consolidation,3769.36
2,C302100003,Approved,24.0,Unmarried,Male,Family,NaN,NaN,Full-Time,7.0,NaN,0.0,Home Improvement,2197.10
3,C302100004,Approved,54.0,Unmarried,Male,Rent,NaN,NaN,Full-Time,105.0,NaN,4.0,Debt Consolidation,1094.45
4,C302100005,Approved,30.0,Unmarried,Female,Family,NaN,NaN,Full-Time,60.0,NaN,1.0,Vacation,3798.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9487,C302109488,Approved,24.0,Unmarried,Female,Family,Professional,Unemployed,Full-Time,13.0,NaN,0.0,Other,3258.78
9488,C302109489,Approved,27.0,Unmarried,Female,Family,Service and Sales Worker,Unemployed,Contract,9.0,NaN,1.0,Debt Consolidation,3634.23
9489,C302109490,Approved,23.0,Married,Female,Own,Armed Forces Occupation,Retiree/Pensioner,Retired,9.0,NaN,0.0,Other,9126.52
9490,C302109491,Approved,22.0,Unmarried,Female,Family,Clerical Support Worker,Unemployed,Full-Time,13.0,NaN,0.0,Other,931.09


In [29]:
contract_data= pd.read_csv("ContractsData.csv")
contract_data.head()

,Contract_ID,Application_ID,Key,Loan_Type,Contract_Start_Date,Expected_Contract_End_Date,Actual_Contract_End_Date,Loan_Term,Total_Loan_Amount
0,CONT100001,C302103426,1,Unsecured Personal Loan,2018-04-12,2023-04-12,NaN,60 - Month(s),24529.89
1,CONT100002,C302103426,2,Unsecured Personal Loan,2018-04-12,2023-04-12,NaN,60 - Month(s),35085.79
2,CONT100003,C302107460,1,Unsecured Personal Loan,202-1-08-31,2024-09-01,NaN,60 - Month(s),53564.88
3,CONT100004,C302107453,1,Unsecured Personal Loan,202-1-08-28,2023-08-28,NaN,48 - Month(s),29073.86
4,CONT100005,C302107447,1,Unsecured Personal Loan,202-1-08-27,2023-08-27,NaN,48 - Month(s),11723.33


In [10]:
contract_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9374 entries, 0 to 9373
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Contract_ID                 9374 non-null   object 
 1   Application_ID              7598 non-null   object 
 2   Key                         9374 non-null   int64  
 3   Loan_Type                   9374 non-null   object 
 4   Contract_Start_Date         9374 non-null   object 
 5   Expected_Contract_End_Date  9374 non-null   object 
 6   Actual_Contract_End_Date    2593 non-null   object 
 7   Loan_Term                   9374 non-null   object 
 8   Total_Loan_Amount           9374 non-null   float64
dtypes: float64(1), int64(1), object(7)
memory usage: 659.2+ KB


In [11]:
contract_data.isnull().sum()

Contract_ID                      0
Application_ID                1776
Key                              0
Loan_Type                        0
Contract_Start_Date              0
Expected_Contract_End_Date       0
Actual_Contract_End_Date      6781
Loan_Term                        0
Total_Loan_Amount                0
dtype: int64

In [14]:
df_contractsnapshot=pd.read_csv("ContractsSnapshotData.csv")
df_contractsnapshot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274371 entries, 0 to 274370
Data columns (total 6 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Contract_ID              274371 non-null  object 
 1   Snapshot Date            274371 non-null  object 
 2   Outstanding Amount       274371 non-null  float64
 3   Due Amount               274371 non-null  float64
 4   Number of Days Past Due  79158 non-null   float64
 5   Contract Status          274371 non-null  object 
dtypes: float64(3), object(3)
memory usage: 12.6+ MB


In [15]:
df_contractsnapshot.isnull().sum()

Contract_ID                     0
Snapshot Date                   0
Outstanding Amount              0
Due Amount                      0
Number of Days Past Due    195213
Contract Status                 0
dtype: int64

In [17]:
df_crb=pd.read_csv("CRB Data.csv")
df_crb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6750 entries, 0 to 6749
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Application_ID    6750 non-null   object
 1   Application_Date  6750 non-null   object
 2   CRB Score         6750 non-null   int64 
 3   CRB Grade         6750 non-null   object
dtypes: int64(1), object(3)
memory usage: 211.1+ KB


In [18]:
df_crb.isnull().sum()

Application_ID      0
Application_Date    0
CRB Score           0
CRB Grade           0
dtype: int64

In [39]:
crb_selected=df_crb.drop(columns=["Application_Date"])
crb_selected

,Application_ID,CRB Score,CRB Grade
0,C302102137,593,D
1,C302106175,625,C
2,C302106175,616,C
3,C302103572,629,C
4,C302100598,685,A
...,...,...,...
6745,C302103644,518,E
6746,C302106169,627,C
6747,C302105619,645,C
6748,C302108238,641,C


In [20]:
df_accounts=pd.read_csv("Current and Savings Account Data.csv")
df_accounts.head(5)

,ID,Account_Number,Application ID,Month (YYYY-MM),Open Date,Closed Date,Average Balance,Overdraft Limit,Number of Days in Debit during Month,Number of Days in Credit during Month,...,Sum of Bounced Cheque transactions,Number of Bounced Cheques,Sum of Debit Card POS Transactions,Number of Debit Card POS Transactions,Sum of Over the Counter Withdrawals,Number of Over the Counter withdrawals,Account Type,Account Status,Number of Joint Holders,SpshotMonth
0,1,ACC10000001,C302100001,2015-09-01,2015-09-06,NaN,1986.09,0.0,0.0,25.0,...,NaN,0.0,NaN,0.0,NaN,0.0,Savings,Active,0.0,2015-09-01
1,2,ACC10000002,C302100001,2015-09-01,2015-09-06,2018-10-04,23.45,NaN,0.0,25.0,...,NaN,0.0,NaN,0.0,NaN,0.0,Savings,Closed,0.0,2015-09-01
2,3,ACC10000001,C302100001,2015-10-01,2015-09-06,NaN,8120.71,0.0,0.0,31.0,...,NaN,0.0,58.62,1.0,NaN,0.0,Savings,Active,0.0,2015-10-01
3,4,ACC10000002,C302100001,2015-10-01,2015-09-06,2018-10-04,25.26,NaN,0.0,31.0,...,NaN,0.0,NaN,0.0,NaN,0.0,Savings,Closed,0.0,2015-10-01
4,5,ACC10000001,C302100001,2015-11-01,2015-09-06,NaN,6624.10,0.0,0.0,30.0,...,NaN,0.0,989.57,3.0,NaN,0.0,Savings,Active,0.0,2015-11-01


In [21]:
df_accounts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 857677 entries, 0 to 857676
Data columns (total 31 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   ID                                      857677 non-null  int64  
 1   Account_Number                          857677 non-null  object 
 2   Application ID                          857672 non-null  object 
 3   Month (YYYY-MM)                         857647 non-null  object 
 4   Open Date                               857647 non-null  object 
 5   Closed Date                             161748 non-null  object 
 6   Average Balance                         857647 non-null  float64
 7   Overdraft Limit                         753484 non-null  float64
 8   Number of Days in Debit during Month    857647 non-null  float64
 9   Number of Days in Credit during Month   857647 non-null  float64
 10  Maximum End of Day Balance              8576

In [22]:
df_accounts.isnull().sum()

ID                                             0
Account_Number                                 0
Application ID                                 5
Month (YYYY-MM)                               30
Open Date                                     30
Closed Date                               695929
Average Balance                               30
Overdraft Limit                           104193
Number of Days in Debit during Month          30
Number of Days in Credit during Month         30
Maximum End of Day Balance                    30
Minimum End of Day Balance                    30
Sum of Monthly Debit Transactions          24686
Sum of Monthly Credit Transactions           148
Last Transaction Date                     332006
Sum of ATM Deposits                       843053
Number of ATM Deposits                        30
Sum of ATM Withdrawals                    668968
Number of ATM Withdrawals                     30
Sum of Cheque Debit Transactions          857043
Number of Cheque Deb

In [56]:
merged_df = df_appli_selected.merge(df_crb,on="Application_ID",how='left')

#merged_df1 = merged_df.merge(df_appli_selected, on="Application ID ", how='left')

In [48]:
frames =[merged_df,df_accounts,crb_selected,df_appli_selected]
df =pd.concat(frames) #concatenating the frames
df

,Contract_ID,Application_ID,Key,Loan_Type,Contract_Start_Date,Expected_Contract_End_Date,Actual_Contract_End_Date,Loan_Term,Total_Loan_Amount,Snapshot Date,...,Gender,Own_Rent_Home,Occupation_Professional_Category,Type_of_Business_Industry_of_Employment,Employment_Status,Time_at_Current_Employment_(Months),Time_Employed_with_Previous_Employer_(Months),Number_of_Dependents,Loan_Purpose,Monthly_Income
0,CONT100001,C302103426,1.0,Unsecured Personal Loan,2018-04-12,2023-04-12,NaN,60 - Month(s),24529.89,2018-04-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CONT100001,C302103426,1.0,Unsecured Personal Loan,2018-04-12,2023-04-12,NaN,60 - Month(s),24529.89,2018-05-31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CONT100001,C302103426,1.0,Unsecured Personal Loan,2018-04-12,2023-04-12,NaN,60 - Month(s),24529.89,2018-06-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CONT100001,C302103426,1.0,Unsecured Personal Loan,2018-04-12,2023-04-12,NaN,60 - Month(s),24529.89,2018-07-31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CONT100001,C302103426,1.0,Unsecured Personal Loan,2018-04-12,2023-04-12,NaN,60 - Month(s),24529.89,2018-08-31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9487,NaN,C302109488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Female,Family,Professional,Unemployed,Full-Time,13.0,NaN,0.0,Other,3258.78
9488,NaN,C302109489,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Female,Family,Service and Sales Worker,Unemployed,Contract,9.0,NaN,1.0,Debt Consolidation,3634.23
9489,NaN,C302109490,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Female,Own,Armed Forces Occupation,Retiree/Pensioner,Retired,9.0,NaN,0.0,Other,9126.52
9490,NaN,C302109491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Female,Family,Clerical Support Worker,Unemployed,Full-Time,13.0,NaN,0.0,Other,931.09


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1026798 entries, 0 to 9491
Data columns (total 60 columns):
 #   Column                                         Non-Null Count   Dtype  
---  ------                                         --------------   -----  
 0   Contract_ID                                    152879 non-null  object 
 1   Application_ID                                 138196 non-null  object 
 2   Key                                            152879 non-null  float64
 3   Loan_Type                                      152879 non-null  object 
 4   Contract_Start_Date                            152879 non-null  object 
 5   Expected_Contract_End_Date                     152879 non-null  object 
 6   Actual_Contract_End_Date                       44506 non-null   object 
 7   Loan_Term                                      152879 non-null  object 
 8   Total_Loan_Amount                              152879 non-null  float64
 9   Snapshot Date                              

In [49]:
df.isnull().sum()

Contract_ID                                       873919
Application_ID                                    888602
Key                                               873919
Loan_Type                                         873919
Contract_Start_Date                               873919
Expected_Contract_End_Date                        873919
Actual_Contract_End_Date                          982292
Loan_Term                                         873919
Total_Loan_Amount                                 873919
Snapshot Date                                     873929
Outstanding Amount                                873929
Due Amount                                        873929
Number of Days Past Due                           998552
Contract Status                                   873929
ID                                                169121
Account_Number                                    169121
Application ID                                    169126
Month (YYYY-MM)                